### **Grid Search**

In [ ]:
#Izgara Arama, modelinizi eğitirken hiperparametrelerinizi manuel olarak ayarlama yerine, aralığınız parametre genişliğindeki tüm yöntemleri sistematik olarak deneyerek ve en iyi performansı veren ayarların otomatik olarak seçilmesini sağlar. Anladığınız gibi, içine istediğiniz modeli ve serbestliği tanıdıktan sonra Grid Search, bu yöntemleri deneyerek sizin için en iyi olanı bulur. Ancak, parametrenin çok olduğu miktarın planlama süresine dikkat edilmesi gerekir.

# Bu yöntem, modelinizi artırmak için güçlü bir araçla birlikte, büyük parametrelerde Rastgele Arama veya Bayesian Optimizasyon gibi kesme alternatif olarak kullanılabilir.

In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_excel("diyabet.xlsx")
df.head(3)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141


In [10]:
df.drop("Y", axis=1, inplace= True)

X = df.drop("S6", axis=1)
y = df["S6"]

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 9)

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#### Grid Search

1. Tahmin edilecek bir algoritma kullan
2. Parametreleri belirle
3. Verileri grid search`e oturt
4. cv unutma
5. puan fonksiyonu belirle

In [19]:
from sklearn.linear_model import ElasticNet

In [21]:
elasticNet = ElasticNet()

In [22]:
from sklearn.model_selection import GridSearchCV

In [27]:
params = {
    "alpha" : [0.01, 0.1, 1, 5, 10, 50, 100 ,1000],
    "l1_ratio" : [0.01, 0.01, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99]
}

In [ ]:
# Aşşağıda yaptığımız şey aslında bir modelin en iyi parametrelerini bulmaya yardımcı olması gibi 
# Estimator kısmına kullanacağımız modeli yazıyor param_grid kısmına yukarıda tanımladığımız elasticNet parametrelerimizi yazıyor altına scoringimiz (Hata kontrolcüsü)onun altına kaç parça bölüneceği verbose ise görüntüleme için help(GridSearchCV) ile görüntüleyebilirsin ne işe yaradığını
gridModel = GridSearchCV(
    estimator= elasticNet,
    param_grid= params,
    scoring= "neg_mean_squared_error",
    cv=5,
    verbose= 2
    )
# Verbose ne işe yarar? : Sonuçta hiçbir şeyi etkilemeyecek sadece görüntüleme kısmında işe yarıyor 

In [29]:
gridModel.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ..........................alpha=0.01, l1_ratio=0.01; total time=   0.0s
[CV] END ...........................alpha=0.01, l1_ratio=0.1; total time=   0.0s
[CV] END ...........................alpha=0.01,

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.01, 0.1, 1, 5, 10, 50, 100, 1000],
                         'l1_ratio': [0.01, 0.01, 0.1, 0.5, 0.7, 0.9, 0.95,
                                      0.99]},
             scoring='neg_mean_squared_error', verbose=2)

In [ ]:
# En iyi sonucu aldığımız değer
gridModel.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=0.01)

In [33]:
gridResult = pd.DataFrame(gridModel.cv_results_)
gridResult.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002799,0.002642,0.000801,0.000401,0.01,0.01,"{'alpha': 0.01, 'l1_ratio': 0.01}",-63.957924,-78.128550,-121.123616,-89.130510,-123.515159,-95.171152,23.571671,6
1,0.002398,0.001001,0.001004,0.001095,0.01,0.01,"{'alpha': 0.01, 'l1_ratio': 0.01}",-63.957924,-78.128550,-121.123616,-89.130510,-123.515159,-95.171152,23.571671,6
2,0.001606,0.000816,0.000598,0.000488,0.01,0.10,"{'alpha': 0.01, 'l1_ratio': 0.1}",-63.974029,-78.161528,-121.092306,-89.091881,-123.576824,-95.179314,23.572583,8


In [35]:
# Burada herhangi bir şekilde git en iyi parametreleri al dememize gerek kalmadı kendisi alıyor zaten
gridPred = gridModel.predict(X_test)

In [36]:
from sklearn.metrics import mean_squared_error

In [ ]:
print("MAE", np.sqrt(mean_squared_error(y_test, gridPred)))

MAE 9.444909715347974


In [38]:
## Hata oranının en düşük olduğu model senin için en iyi model değildir